# Exercise with Natural Language Processing

For todays exersice we will be doing two things.  The first is to build the same model with the same data that we did in the lecture, the second will be to build a new model with new data. 

## PART 1: 
- 20 Newsgroups Corpus


## PART 2:
- Republican vs Democrat Tweet Classifier

In [1]:
# Import pandas for data handling
import pandas as pd

# NLTK is our Natural-Language-Took-Kit
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Libraries for helping us with strings
import string
# Regular Expression Library
import re

# Import our text vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


# Import our classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier


# Import some ML helper function
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# Import our metrics to evaluate our model
from sklearn import metrics
from sklearn.metrics import classification_report


# Library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# You may need to download these from nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mahernigar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mahernigar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mahernigar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Load and display data.
1. Load the 20-newsgroups.csv data into a dataframe.
1. Print the shape
1. Inspect / remove nulls and duplicates
1. Find class balances, print out how many of each topic_category there are.

In [2]:
# 1. Load the 20-newsgroups.csv data into a dataframe.
df = pd.read_csv('data/20-newsgroups.csv')
# 2. Print the shape
print(df.shape)
df.head()

(11314, 4)


,id,message,topic,topic_category
0,0,From: lerxst@wam.umd.edu (where's my thing)\r\...,7,rec.autos
1,1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,3,From: jgreen@amber (Joe Green)\r\nSubject: Re:...,1,comp.graphics
4,4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [3]:
# 3. Inspect / remove nulls and duplicates
print(df.isnull().sum())


id                0
message           0
topic             0
topic_category    0
dtype: int64


In [4]:
#For duplicates
print(df.duplicated().sum())


0


In [5]:
# 4. Find class balances, print out how many of each topic_category there are.
#counting how many in each of these categories
df.topic_category.value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
sci.med                     594
rec.autos                   594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: topic_category, dtype: int64

# Text Pre-Processing 
(aka Feature engineering)
1. Make a function that makes all text lowercase.
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
2. Make a function that removes all punctuation. 
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
3. Make a function that removes all stopwords.
    * Do a sanity check by feeding in a test sentence into the function. 
    
    
4. EXTRA CREDIT (This step only): Make a function that stemms all words. 


5. Mandatory: Make a pipeline function that applys all the text processing functions you just built.
    * Do a sanity check by feeding in a test sentence into the pipeline. 
    
    
    
6. Mandatory: Use `df['message_clean'] = df[column].apply(???)` and apply the text pipeline to your text data column. 

In [6]:
# 1. Make a function that makes all text lowercase.
test_string = 'This is A SENTENCE with LOTS OF CAPS.'

def lower_Case(a_string):
    return a_string.lower()

lower_Case(test_string)

'this is a sentence with lots of caps.'

In [7]:
# 2. Make a function that removes all punctuation. 
def remove_punc(my_string):
    my_string = re.sub(r'[^\w\s]','',my_string)
    return my_string 
    

test_string = 'This is a sentence! 50 With lots of punctuation??? & other #things.'
remove_punc(test_string)

'This is a sentence 50 With lots of punctuation  other things'

In [8]:
# 3. Make a function that removes all stopwords.


def remove_stopwords(a_string):
    #Separate the sentence in words
    words = word_tokenize(a_string)
    
    #Creating an empty list
    valid = []
    
    #If the word is in our list, but not in stopwords, we add it to our valid list!
    for word in words:
        if word not in stopwords:
            valid.append(word)
            
    # putting the valid words into a string
    a_string = ' '.join(valid)
    return a_string
    
test_string = 'This is a sentence! With some different stopwords i have added in here.'  
remove_stopwords(test_string)


'This sentence ! With different stopwords added .'

In [9]:
# 4. EXTRA CREDIT: Make a function that stemms all words. 
#Stemming is the process of reducing a word to its word stem 

def stemms(a_string):
    
# Initalize our Stemmer
     porter = PorterStemmer()
    
     # Separating the sentence into a list of words
     words = word_tokenize(a_string)
    
     # Make a list to append valid words into
     valid = []

     # Loop through all the words
     for word in words:
        # Stem the word
         stemms_word = porter.stem(word)
        
         # Append stemmed word into our new valid list
         valid.append(stemms_word)
        
     # Join the list of words together into a string
     a_string = ' '.join(valid)

     return a_string 


a_sentence = 'I played and started playing with players and we all love to play with plays'
stemms(a_sentence)



'I play and start play with player and we all love to play with play'

In [10]:
# 5. MANDATORY: Make a pipeline function that applys all the text processing functions you just built.

def pipeline(input_string):
    input_string = lower_Case(input_string)
    input_string = remove_punc(input_string)
    input_string = remove_stopwords(input_string)    
    input_string = stemms(input_string)
    return input_string


test_string = 'I played and started playing with players and we all love to play with plays'
pipeline(test_string)





'play start play player love play play'

In [13]:
# 6. Mandatory: Use `df[column].apply(???)` and apply the text pipeline to your text data column. 


#creating a new column
df['message_clean'] = df['message']

#We are cleaning the sentences using our new method called pipeline
df['message_clean'] = df['message'].apply(pipeline)
print("ORIGINAL TEXT:", df['message'][0])
print("CLEANDED TEXT:", df['message_clean'][0])




ORIGINAL TEXT: From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----





CLEANDED TEXT: lerxstwamumdedu where thing subject car nntppostinghost rac3wamumdedu organ univers maryland colleg park line 15 wonder anyon could enlighten car saw day 2door sport car look late 60 earli 70 call bricklin door realli small addit front bumper separ

In [14]:
df.head()

,id,message,topic,topic_category,message_clean
0,0,From: lerxst@wam.umd.edu (where's my thing)\r\...,7,rec.autos,lerxstwamumdedu where thing subject car nntppo...
1,1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware,guykuocarsonuwashingtonedu guy kuo subject si ...
2,2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware,twillisececnpurdueedu thoma e willi subject pb...
3,3,From: jgreen@amber (Joe Green)\r\nSubject: Re:...,1,comp.graphics,jgreenamb joe green subject weitek p9000 organ...
4,4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space,jcmheadcfaharvardedu jonathan mcdowel subject ...


# Text Vectorization

1. Define your `X` and `y` data. 


2. Initialize a vectorizer (you can use TFIDF or BOW, it is your choice).
    * Do you want to use n-grams..?


3. Fit your vectorizer using your X data.
    * Remember, this process happens IN PLACE.


4. Transform your X data using your fitted vectorizer. 
    * `X = vectorizer.???`



5. Print the shape of your X.  How many features (aka columns) do you have?

In [15]:


# 1. Define your `X` and `y` data. 
X = df['message_clean'].values

y = df['topic_category'].values



In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 2. Initialize a vectorizer (you can use TFIDF or BOW, it is your choice).

vectorizer = TfidfVectorizer()

In [19]:
# 3. Fit your vectorizer using your X data
vectorizer.fit(X)


TfidfVectorizer()

In [26]:
# 4. Transform your X data using your fitted vectorizer. 

X_data = vectorizer.transform(X)
features = vectorizer.get_feature_names()
weights = vectorizer.idf_

print(len(features), len(weights))

df_idf = pd.DataFrame.from_dict( {'feature': features, 'idf': weights})

df_idf = df_idf.sort_values(by='idf', ascending=False)

df_idf

119708 119708


,feature,idf
59854,konstantinopoli,9.640737
70532,maxutu72pl707u7vznlznkjznki8f727b9nevz,9.640737
70529,maxson,9.640737
70528,maxslinkynyuedu,9.640737
70527,maxslinkycsnyuedu,9.640737
...,...,...
23975,articl,1.818293
116644,write,1.623100
87442,organ,1.038284
62061,line,1.002832


In [27]:
# 5. Print the shape of your X.  How many features (aka columns) do you have?

print(X_data.shape)

(11314, 119708)


# Split your data into Training and Testing data. 

In [31]:
# Split our data into testing and training like always. 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Save the raw text for later just incase
#X_train_text = X_train
#X_test_text = X_test

___
# Build and Train Model
Use Multinomial Naive Bayes to classify these documents. 

1. Initalize an empty model. 
2. Fit the model with our training data.


Experiment with different alphas.  Use the alpha gives you the best result.

EXTRA CREDIT:  Use grid search to programmatically do this for you. 

In [32]:
#The Multinomial Naive Bayes algorithm is a Bayesian learning approach popular in Natural Language Processing (NLP).
#The program guesses the tag of a text, such as an email or a newspaper story, using the Bayes theorem.

# 1. Initalize an empty model. 
model = MultinomialNB(alpha=.05)



In [33]:
# Fit our model with our training data.
model.fit(X_train, y_train)




# Evaluate the model.

1. Make new predicitions using our test data. 
2. Print the accuracy of the model. 
3. Print the confusion matrix of our predictions. 
4. Using `classification_report` print the evaluation results for all the classes. 



In [79]:
# 1. Make new predictions of our testing data. 

y_pred = model.predict(X_test)



In [ ]:
# 2. Print the accuracy of the model. 
accuracy = model.score(X_test, y_test)

print("Model Accuracy: %f" % accuracy)

In [81]:
# 3. Plot the confusion matrix of our predictions
#confusion matrix
classification_report(y_test, y_pred, target_names=model.classes_)


In [ ]:
# 4. Using `classification_report` print the evaluation results for all the classes. 

print(classification_report(y_test, y_pred, target_names=model.classes_))

# Manual predicition
Write a new sentence that you think will be classified as talk.politics.guns. 
1. Apply the text pipeline to your sentence
2. Transform your cleaned text using the `X = vectorizer.transform([your_text])`
    * Note, the `transform` function accepts a list and not a individual string.
3. Use the model to predict your new `X`. 
4. Print the prediction

In [84]:
my_sentence = "limiting access to guns will save lives and reduce crime"

# 1. Apply the text pipeline to your sentence
piped_sentence = pipeline(my_sentence)

# 2. Transform your cleaned text using the `X = vectorizer.transform([your_text])`\
X = vectorizer.transform([piped_sentence])

# 3. Use the model to predict your new `X`. 
model.predict(X)
# 4. Print the prediction


___
# PART 2: Twitter Data
This part of the exercise is un-guided on purpose.  

Using the `dem-vs-rep-tweets.csv` build a classifier to determine if a tweet was written by a democrat or republican. 

Can you get an f1-score higher than %82

In [95]:
# 1. Load the 20-newsgroups.csv data into a dataframe.
# 2. Print the shape
df = pd.read_csv('data/dem-vs-rep-tweets.csv')
print(df.shape)
df.head()


(86460, 3)


,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


In [96]:
#checking for null
print(df.isnull().sum())

Party     0
Handle    0
Tweet     0
dtype: int64


In [97]:
#checking for duplicates
print(df.duplicated().sum())

57


In [99]:
#Drop duplicates
df = df.drop_duplicates()
df.shape

(86403, 3)

In [101]:
df.head()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...


In [102]:
# 4. Find class balances, print out how many of each topic_category there are.
#counting how many in each of these categories
df.Party.value_counts()

Republican    44362
Democrat      42041
Name: Party, dtype: int64

In [108]:
def pipeline(input_string):
    input_string = lower_Case(input_string)
    input_string = remove_punc(input_string)
    input_string = remove_stopwords(input_string)    
    input_string = stemms(input_string)
    return input_string


#test_string = 'I played and started playing with players and we all love to play with plays'
#pipeline(test_string)



In [111]:
#creating a new column
df['t_clean'] = df['Tweet']

#We are cleaning the sentences using our new method called pipeline
df['t_clean'] = df['Tweet'].apply(pipeline)




In [112]:

df.head()

,Party,Handle,Tweet,message_clean,t_clean
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P...",today senat dem vote savetheinternet proud sup...,today senat dem vote savetheinternet proud sup...
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...,rt winterhavensun winter resid alta vista teac...,rt winterhavensun winter resid alta vista teac...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...,rt nbclatino repdarrensoto note hurrican maria...,rt nbclatino repdarrensoto note hurrican maria...
3,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...,rt nalcabpolici meet repdarrensoto thank take ...,rt nalcabpolici meet repdarrensoto thank take ...
4,Democrat,RepDarrenSoto,RT @Vegalteno: Hurricane season starts on June...,rt vegalteno hurrican season start june 1st pu...,rt vegalteno hurrican season start june 1st pu...


In [113]:
X = df['t_clean'].values
y = df['Party'].values

In [114]:
# Split our data into testing and training like always. 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


# Save the raw text for later just incase
X_train_text = X_train
X_test_text = X_test

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer


# Initialize our vectorizer
vectorizer = TfidfVectorizer()

# This makes your vocab matrix
vectorizer.fit(X_train)

# This transforms your documents into vectors.
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

print(X_train.shape, type(X))

(69122, 101702) <class 'numpy.ndarray'>


In [127]:
features = vectorizer.get_feature_names()
weights = vectorizer.idf_

print(len(features), len(weights))

df_idf = pd.DataFrame.from_dict( {'feature': features, 'idf': weights})

df_idf = df_idf.sort_values(by='idf', ascending=False)

df_idf

101702 101702


,feature,idf
50851,httpstcomjulusrbwu,11.450496
54390,httpstcoolqacj0dt2,11.450496
54397,httpstcooltr8qvnz8,11.450496
54396,httpstcooltmasoma4,11.450496
54395,httpstcoolthxcfpo3,11.450496
...,...,...
100882,work,3.973740
95916,thank,3.581094
4622,amp,3.348818
96634,today,3.311055


In [131]:
# 1. Initalize an empty model. 
model = MultinomialNB(alpha=.05)
# Fit our model with our training data.
model.fit(X_train, y_train)





MultinomialNB(alpha=0.05)

In [132]:
# 1. Make new predictions of our testing data. 

y_pred = model.predict(X_test)

# 2. Print the accuracy of the model. 
accuracy = model.score(X_test, y_test)

print("Model Accuracy: %f" % accuracy)


Model Accuracy: 0.815694


In [140]:
f1score = f1_score(y_test, y_pred, pos_label='Democrat')
print(f'Democrats: {f1score}')

Democrats: 0.8073549869957055


In [141]:
f1score = f1_score(y_test, y_pred, pos_label='Republican')
print(f'Republicans: {f1score}')

Republicans: 0.8233401741638471
